In [ ]:
import numpy as np
import pandas as pd

import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
tqdm.pandas()

import tqdm.notebook as tq

import os

from keras import models
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [ ]:
# data = pd.read_csv('/content/drive/MyDrive/codeep-modeling/dataset/1220_data.csv')
# len(data['y'].unique()) #3423

# 전처리 함수

In [ ]:
def preprocessed(books, path):

  new_image = []
  no_image = []

  for i in tq.tqdm(range(len(books[:100]))):
    book_file = path + books[i]
  
    try:
      image = load_img(book_file, target_size = (224, 224))
      image = img_to_array(image)
      image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
      image = preprocess_input(image)
      new_image.append(image)
    except:
      no_image.append(book_file)
      print(book_file, end = ' ')
  
  return new_image, no_image

# feature extraction 함수

In [ ]:
def get_features(model, image_list):
  extracted_image = []
  
  for i in tq.tqdm(range(len(image_list))):
    features = model.predict(image_list[i], verbose=0)
    extracted_image.append(features)
  
  return extracted_image

In [ ]:
books = os.listdir('/content/drive/MyDrive/Colab Notebooks/cover')
path = '/content/drive/MyDrive/Colab Notebooks/cover'

OSError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# VGG-16

In [ ]:
from keras.applications.vgg16 import VGG16

base_model = VGG16(weights='imagenet')
model_VGG16 = models.Model(inputs=base_model.input, outputs=base_model.get_layer('flatten').output)
model_VGG16.summary()
# model_VGG16.trainable = True

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
from keras.applications.vgg16 import preprocess_input

vgg_preprocessed_images, vgg_no_image = preprocessed(books, path)
vgg_extracted_images = get_features(model_VGG16, vgg_preprocessed_images)

In [ ]:
vgg_extracted_images

[array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         3.6234643]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0.       , 0.       , 0.       , ..., 0.       , 3.2376344,
         0.       ]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[ 0.      , 19.861729,  0.      , ...,  0.      ,  0.      ,
          0.      ]], dtype=float32),
 array([[0.      , 7.836347, 0.      , ..., 0.      , 0.      , 0.      ]],
       dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0.    , 0.    , 0.    , ..., 0.    , 4.4356, 0.    ]],
       dtype

# efficientNet

In [ ]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input

base_model = EfficientNetB0(weights='imagenet')
model_eff = models.Model(inputs = base_model.input, outputs = base_model.get_layer('avg_pool').output)
model_eff.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_14 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 224, 224, 3)  0           input_14[0][0]                   
__________________________________________________________________________________________________
normalization_1 (Normalization) (None, 224, 224, 3)  7           rescaling_1[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 225, 225, 3)  0           normalization_1[0][0]            
____________________________________________________________________________________________

In [ ]:
eff_preprocessed_images, eff_no_image = preprocessed(books, path)
eff_extracted_images = get_features(model_eff, eff_preprocessed_images)

In [ ]:
eff_extracted_images

[array([[ 0.624452  , -0.18639588, -0.06445576, ..., -0.08309857,
         -0.11428856,  0.41255975]], dtype=float32),
 array([[ 0.23043904,  0.01360821,  0.00540246, ...,  0.10838333,
         -0.05766474,  0.32507455]], dtype=float32),
 array([[ 0.9852869 , -0.14257617, -0.07796369, ..., -0.16231489,
         -0.01775704,  0.6259939 ]], dtype=float32),
 array([[-0.15037382, -0.06602579, -0.0560917 , ..., -0.09219427,
          0.03682436,  0.08589005]], dtype=float32),
 array([[ 0.15288343,  0.06913911, -0.05998283, ...,  0.04298245,
         -0.11518764,  0.9383688 ]], dtype=float32),
 array([[ 0.3333407 , -0.11453611, -0.11862303, ..., -0.08881133,
         -0.12700477,  0.11446132]], dtype=float32),
 array([[ 0.51039344, -0.13099487, -0.11432562, ...,  0.12678488,
         -0.12787239,  0.59235686]], dtype=float32),
 array([[-0.1678035 ,  0.24109165,  0.20027868, ...,  0.05311074,
          0.1696281 ,  0.50104177]], dtype=float32),
 array([[ 1.1109064 , -0.12907924, -0.06872717, 

# mobileNet

In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet, preprocess_input

base_model = MobileNet(weights='imagenet')
model_mm = models.Model(inputs = base_model.input, outputs = base_model.get_layer('global_average_pooling2d_11').output)
model_mm.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0   

In [ ]:
mm_preprocessed_images, mm_no_image = preprocessed(books, path)
mm_extracted_images = get_features(model_mm, mm_preprocessed_images)

In [ ]:
mm_extracted_images

[array([[0.09474745, 0.32360792, 1.6770306 , ..., 0.36893874, 1.0162205 ,
         0.1364258 ]], dtype=float32),
 array([[0.4277826 , 0.7573448 , 0.45240545, ..., 0.1771332 , 0.20586467,
         0.5657719 ]], dtype=float32),
 array([[0.16486669, 0.02834236, 2.6596413 , ..., 0.        , 0.2603526 ,
         1.4319829 ]], dtype=float32),
 array([[0.16268148, 0.16342463, 2.049921  , ..., 0.07689057, 0.34688148,
         0.44950813]], dtype=float32),
 array([[0.        , 0.06401356, 2.0749984 , ..., 0.        , 0.23374729,
         1.4064199 ]], dtype=float32),
 array([[0.01005215, 0.0337877 , 1.4502171 , ..., 0.9333454 , 0.26558113,
         0.76825994]], dtype=float32),
 array([[1.2268912 , 0.39459696, 0.5944767 , ..., 0.        , 2.1401603 ,
         0.42094874]], dtype=float32),
 array([[0.12140343, 0.25661206, 0.07833146, ..., 1.2601314 , 0.635751  ,
         0.18463957]], dtype=float32),
 array([[0.15813705, 0.83636796, 1.9913489 , ..., 0.        , 1.5825225 ,
         0.50202084]],